In [1]:
import math

import numpy as np

max_n_cars_at_first_location = 20
max_n_cars_at_second_location = 20
max_n_cars_to_move = 5
lambda_requests_at_first_location = 3
lambda_requests_at_second_location = 4
lambda_returns_at_first_location = 3
lambda_returns_at_second_location = 2
gamma = 0.9
theta = 1

def poisson_probability(lambduh, n, max_n):
    assert lambduh > 0
    assert 0 <= n <= max_n

    if n == max_n:
        return 1 - sum([poisson_probability(lambduh, i, max_n) for i in range(max_n)])
    else:
        return lambduh ** n * math.exp(-lambduh) / math.factorial(n)

def new_state_value(n_cars_at_first_location, n_cars_at_second_location, v, a):
    assert 0 <= n_cars_at_first_location <= max_n_cars_at_first_location
    assert 0 <= n_cars_at_second_location <= max_n_cars_at_second_location
    assert a <= n_cars_at_first_location and -a <= n_cars_at_second_location

    morning_n_cars_at_first_location = min(n_cars_at_first_location - a, max_n_cars_at_first_location)
    max_n_cars_returned_at_first_location = max_n_cars_at_first_location - morning_n_cars_at_first_location
    morning_n_cars_at_second_location = min(n_cars_at_second_location + a, max_n_cars_at_second_location)
    max_n_cars_returned_at_second_location = max_n_cars_at_second_location - morning_n_cars_at_second_location

    new_value = 0.0
    for n_cars_requested_at_first_location in range(morning_n_cars_at_first_location + 1):
        for n_cars_returned_at_first_location in range(max_n_cars_returned_at_first_location + 1):
            for n_cars_requested_at_second_location in range(morning_n_cars_at_second_location + 1):
                for n_cars_returned_at_second_location in range(max_n_cars_returned_at_second_location + 1):
                    p_requests_at_first_location = poisson_probability(lambda_requests_at_first_location, n_cars_requested_at_first_location, morning_n_cars_at_first_location)
                    p_returns_at_first_location = poisson_probability(lambda_returns_at_first_location, n_cars_returned_at_first_location, max_n_cars_returned_at_first_location)
                    p_requests_at_second_location = poisson_probability(lambda_requests_at_second_location, n_cars_requested_at_second_location, morning_n_cars_at_second_location)
                    p_returns_at_second_location = poisson_probability(lambda_returns_at_second_location, n_cars_returned_at_second_location, max_n_cars_returned_at_second_location)
                    p = p_requests_at_first_location * p_returns_at_first_location * p_requests_at_second_location * p_returns_at_second_location
                    r = 10 * (n_cars_requested_at_first_location + n_cars_requested_at_second_location) - 2 * abs(a)
                    final_n_cars_at_first_location = morning_n_cars_at_first_location + n_cars_returned_at_first_location - n_cars_requested_at_first_location
                    final_n_cars_at_second_location = morning_n_cars_at_second_location + n_cars_returned_at_second_location - n_cars_requested_at_second_location
                    new_value += p * (r + gamma * v[final_n_cars_at_first_location, final_n_cars_at_second_location])
    return new_value

def update_values(v, pi):
    for n_cars_at_first_location in range(max_n_cars_at_first_location + 1):
        for n_cars_at_second_location in range(max_n_cars_at_second_location + 1):
            a = pi[n_cars_at_first_location, n_cars_at_second_location]
            v[n_cars_at_first_location, n_cars_at_second_location] = new_state_value(n_cars_at_first_location, n_cars_at_second_location, v, a)

def update_policy(v, pi):
    for n_cars_at_first_location in range(max_n_cars_at_first_location + 1):
        for n_cars_at_second_location in range(max_n_cars_at_second_location + 1):
            possible_actions = [a for a in range(-max_n_cars_to_move, max_n_cars_to_move + 1) if a <= n_cars_at_first_location and -a <= n_cars_at_second_location]
            pi[n_cars_at_first_location, n_cars_at_second_location] = possible_actions[np.argmax([new_state_value(n_cars_at_first_location, n_cars_at_second_location, v, a) for a in possible_actions])]

# Initialization
v = np.zeros((max_n_cars_at_first_location + 1, max_n_cars_at_second_location + 1))
pi = np.zeros((max_n_cars_at_first_location + 1, max_n_cars_at_second_location + 1), dtype=int)
                                         
# Policy Evaluation
def evaluate_policy(v, pi):
    delta = float('inf')
    while delta >= theta:
        old_v = np.copy(v)
        update_values(v, pi)
        delta = abs(v - old_v).max()
        print('v:\n', v[::-1])
        print('delta:', delta)

# Policy Improvement
def improve_policy(v, pi):
    policy_stable = False
    while not policy_stable:
        print('v:\n', v[::-1])
        print('pi:\n', pi[::-1])
        evaluate_policy(v, pi)
        old_pi = np.copy(pi)
        update_policy(v, pi)
        if (pi == old_pi).all():
            policy_stable = True

improve_policy(v, pi)
print('final v:\n', v[::-1])
print('final pi:\n', pi[::-1])

v:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0

v:
 [[186.98199432 197.43974246 208.31709298 219.1056273  229.5164605
  239.50440829 249.01318523 257.93739388 266.18802009 273.72216607
  280.53511987 286.64497875 292.0832463  296.89122305 301.11782365
  304.81444019 308.02429138 310.76299159 312.97194062 314.46641117
  315.1085425 ]
 [176.47977401 187.87229015 200.47419251 212.68894207 223.77789089
  233.94578998 243.43327923 252.2682889  260.39985711 267.79503275
  274.45428397 280.40075663 285.67125476 290.31278461 294.3820747
  297.94866941 301.11092278 304.01620624 306.75653148 309.0341194
  310.25485504]
 [161.53439501 173.55968506 187.3411592  200.54504716 212.09014549
  222.33266572 231.71650697 240.3633446  248.25078288 255.35721727
  261.69341959 267.29391839 272.20718541 276.49171804 280.21687843
  283.47315694 286.41076467 289.29310845 292.34644005 295.19309783
  296.86386369]
 [148.94989217 160.9404773  174.65587305 187.78851333 199.23635947
  209.32478352 218.48650961 226.84451295 234.38374376 241.09430247
  247.0015369

v:
 [[288.87980655 299.14859479 309.64477291 320.03818861 330.13770546
  339.91700229 349.35876409 358.42184671 367.07231401 375.29654424
  383.09475038 390.47158104 397.43037519 403.96960156 410.07687701
  415.71448596 420.79122207 425.12549305 428.43237246 430.42505322
  431.13835442]
 [282.72711115 293.53319189 305.0195176  316.23704882 326.74550403
  336.66373681 346.14343861 355.21739206 363.87068653 372.09243234
  379.88258244 387.24537937 394.18411481 400.69784937 406.77629956
  412.38904211 417.46923123 421.89454365 425.44902008 427.79832912
  428.77448335]
 [273.7724887  284.97321336 297.18603615 309.00895222 319.82545783
  329.8606796  339.38495161 348.48059269 357.14372674 365.36384784
  373.13987132 380.47534326 387.37349279 393.83428994 399.85025864
  405.39879777 410.43508632 414.88627288 418.59326692 421.18926323
  422.35409591]
 [265.37431163 276.62658542 288.93868348 300.85479946 311.7348215
  321.81328415 331.36989853 340.48754416 349.15952802 357.37297859
  365.12554

v:
 [[352.60194915 362.7621084  373.02670918 383.14330799 392.95179823
  402.41502603 411.52065835 420.25234713 428.60164573 436.57243323
  444.17574046 451.42346679 458.32383412 464.87690078 471.06620424
  476.84074443 482.0820151  486.56354662 489.94972326 491.93996383
  492.61819191]
 [348.70952404 359.20689792 370.09341033 380.728647   390.79586581
  400.3492705  409.48316326 418.22803547 426.58771301 434.5680519
  442.18009202 449.43552239 456.34240704 462.90080763 469.09483647
  474.87660455 480.13979067 484.68902113 488.22478898 490.41898556
  491.24459785]
 [342.99913786 353.74118176 365.07886881 376.09179732 386.35371522
  395.98522758 405.15529597 413.9260502  422.30887553 430.3104318
  437.94150129 445.21348754 452.13424734 458.70381121 464.90680604
  470.69788135 475.97994595 480.58039221 484.22562892 486.56721786
  487.49851029]
 [337.50119696 348.27538612 359.67499812 370.74669164 381.05106604
  390.71581408 399.91666929 408.71701149 417.12730649 425.1534072
  432.8057294

v:
 [[393.40628487 403.50013186 413.62255556 423.56676864 433.18926964
  442.44353761 451.3171466  459.80543238 467.91183711 475.64721035
  483.02569098 490.06067853 496.7615072  503.12910977 509.14732558
  514.76453397 519.8605088  524.20655495 527.46902878 529.36305937
  529.9936821 ]
 [390.89119436 401.20158556 411.72346314 422.00096067 431.78984827
  441.10226435 449.99437323 458.49169544 466.60564186 474.34841421
  481.73423615 488.77643974 495.48430003 501.85874606 507.88389447
  513.50987174 518.62371139 523.01559093 526.37374682 528.39588259
  529.11844215]
 [387.16882936 397.6357076  408.44627732 418.96568324 428.87947634
  438.24229648 447.15825916 455.67331263 463.8038732  471.56282343
  478.96438619 486.02185967 492.74449515 499.13322993 505.17239043
  510.81334374 515.94816842 520.3800138  523.81174819 525.92777282
  526.71615344]
 [383.50395339 393.99133647 404.84145004 415.39850401 425.33973589
  434.72442815 443.66117675 452.19690216 460.34762072 468.12589809
  475.5459

...

v:
 [[545.98506073 552.19177176 557.75759161 562.86550835 567.64443085
  572.17104767 576.48797102 580.6215366  584.59050307 588.40169505
  592.0322488  595.3993324  598.32557292 600.5431138  603.83324374
  606.7831125  609.36808958 611.63276847 613.69930511 614.84687454
  615.21867475]
 [542.2461214  548.60149075 554.33412468 559.61829028 564.57822828
  569.2892783  573.79390145 578.11717777 582.27443369 586.26766015
  590.06893254 593.589685   596.64443195 598.95518617 602.52897234
  605.81135188 608.75569638 611.33991042 613.60721101 614.76195459
  615.13755422]
 [538.11538668 544.63514712 550.55275654 556.03275014 561.19379286
  566.10902545 570.81982115 575.34896061 579.70730175 583.89114238
  587.86708204 591.5412103  594.7213151  598.62657651 600.93775084
  604.50023938 607.77133777 610.70673706 613.28372891 614.45452003
  614.83624852]
 [533.681742   540.38093368 546.50054802 552.19384618 557.57237077
  562.70572148 567.63325964 572.37517152 576.93809487 581.31292243
  585.4607

v:
 [[549.85202175 556.31366948 562.16224052 567.55689646 572.60678378
  577.22663349 581.82836645 586.18607901 590.31571061 594.22125479
  597.89439956 601.30550745 604.59537547 607.58063123 610.10891896
  612.79678853 615.11383633 617.2031427  619.02290643 620.06254007
  620.39773004]
 [546.01197391 552.63317469 558.66072733 564.10615477 569.49871773
  574.54635142 579.12946208 583.72761239 588.08225054 592.20944881
  596.11328187 599.78544037 603.19620764 606.45291487 609.43913106
  611.97051105 614.65056062 616.98042029 618.81829387 619.87864064
  620.22769928]
 [541.78304163 548.57580571 554.56875144 560.59430868 565.99352935
  571.38088594 576.42335465 580.95282264 585.54397189 589.89221175
  594.01392348 597.91345117 601.58263066 604.99174581 608.19989285
  611.1874117  613.72456178 616.37403617 618.24074858 619.32482587
  619.68670253]
 [537.25189095 544.22334053 550.51146992 556.45968894 562.47988756
  567.82534203 573.20405316 578.23797151 582.71176822 587.29253401
  591.6309

v:
 [[551.56861647 558.10091693 564.03386451 569.52288326 574.67368601
  579.30261337 584.00535317 588.46506764 592.69523582 596.69728064
  600.46029476 603.95209409 607.22343651 610.27369862 612.85297425
  615.51625928 617.81430208 619.89142519 621.73940822 622.79093632
  623.1276399 ]
 [547.72255461 554.41623615 560.53063263 566.00956445 571.49784728
  576.64775254 581.26094726 585.96205837 590.42019451 594.64891208
  598.64970446 602.41171349 605.90278772 609.1589762  612.20878649
  614.78898256 617.44879902 619.75284423 621.6117419  622.67444932
  623.01816043]
 [543.48022661 550.34622793 556.38792122 562.50183949 567.96088614
  573.44735844 578.59532488 583.18563253 587.88380693 592.33896409
  596.56477635 600.56286748 604.32250285 607.81165383 611.04611544
  614.09468409 616.6761095  619.32241834 621.2047239  622.28432265
  622.63594665]
 [538.92977252 545.97379226 552.31780496 558.339982   564.45229279
  569.88810146 575.37162895 580.51651374 585.08288777 589.7770232
  594.22797

v:
 [[552.50057145 559.04959774 565.00376407 570.5179933  575.69743464
  580.32137366 585.05501973 589.54793428 593.81299862 597.85087613
  601.64971356 605.17614213 608.43532121 611.51501593 614.11884342
  616.76848308 619.05708972 621.12855711 622.99050464 624.04830356
  624.38598425]
 [548.66527525 555.37583061 561.51175956 566.9912345  572.50530133
  577.68448936 582.30065961 587.03379437 591.52619532 595.79077518
  599.82822806 603.62672428 607.15291296 610.40497723 613.48455484
  616.08887276 618.73687551 621.02826681 622.89821002 623.96283179
  624.30427939]
 [544.43235015 551.31508934 557.36079664 563.49676286 568.96598998
  574.47957149 579.65820263 584.26297194 588.99520513 593.48664724
  597.75025554 601.78677919 605.58444466 609.10996064 612.35173364
  615.430949   618.03599897 620.67758548 622.56969271 623.64842854
  623.9954979 ]
 [539.89007081 546.95035606 553.29984234 559.3352149  565.47083693
  570.9279762  576.44074382 581.61849584 586.21106689 590.94210534
  595.4322

v:
 [[552.93290284 559.49842689 565.46442831 570.98363497 576.16124769
  581.05773881 585.70380398 590.11280574 594.28723819 598.28537366
  602.07094509 605.58758553 608.84963683 611.92160056 614.62800568
  617.17059522 619.45833915 621.5289642  623.38763413 624.44350628
  624.78039494]
 [549.07479626 555.80078093 561.94788077 567.6580168  573.02845802
  578.14536907 583.04237842 587.68982438 592.09984624 596.27489198
  600.26594522 604.05166146 607.56826682 610.82425331 613.89623217
  616.60041289 619.14526683 621.43498397 623.30131748 624.36325427
  624.70327359]
 [544.81973423 551.71596151 558.05406869 563.96451687 569.63830159
  575.00899687 580.1123228  585.00941087 589.6595681  594.0720098
  598.248855   602.22900983 606.01566697 609.53265388 612.77962891
  615.85191844 618.5500255  621.09664727 622.98531952 624.06100982
  624.40619854]
 [540.25970323 547.33074348 553.86370458 560.03439876 565.94500555
  571.60461677 576.97548741 582.06276749 586.95855134 591.6114978
  596.027227

In [3]:
import math

import numpy as np

max_n_cars_at_first_location = 20
max_n_cars_at_second_location = 20
max_n_cars_to_move = 5
lambda_requests_at_first_location = 3
lambda_requests_at_second_location = 4
lambda_returns_at_first_location = 3
lambda_returns_at_second_location = 2
gamma = 0.9
theta = 1

def poisson_probability(lambduh, n, max_n):
    assert lambduh > 0
    assert 0 <= n <= max_n

    if n == max_n:
        return 1 - sum([poisson_probability(lambduh, i, max_n) for i in range(max_n)])
    else:
        return lambduh ** n * math.exp(-lambduh) / math.factorial(n)

def new_state_value(n_cars_at_first_location, n_cars_at_second_location, v, a):
    assert 0 <= n_cars_at_first_location <= max_n_cars_at_first_location
    assert 0 <= n_cars_at_second_location <= max_n_cars_at_second_location
    assert a <= n_cars_at_first_location and -a <= n_cars_at_second_location

    morning_n_cars_at_first_location = min(n_cars_at_first_location - a, max_n_cars_at_first_location)
    max_n_cars_returned_at_first_location = max_n_cars_at_first_location - morning_n_cars_at_first_location
    morning_n_cars_at_second_location = min(n_cars_at_second_location + a, max_n_cars_at_second_location)
    max_n_cars_returned_at_second_location = max_n_cars_at_second_location - morning_n_cars_at_second_location

    new_value = 0.0
    for n_cars_requested_at_first_location in range(morning_n_cars_at_first_location + 1):
        for n_cars_returned_at_first_location in range(max_n_cars_returned_at_first_location + 1):
            for n_cars_requested_at_second_location in range(morning_n_cars_at_second_location + 1):
                for n_cars_returned_at_second_location in range(max_n_cars_returned_at_second_location + 1):
                    p_requests_at_first_location = poisson_probability(lambda_requests_at_first_location, n_cars_requested_at_first_location, morning_n_cars_at_first_location)
                    p_returns_at_first_location = poisson_probability(lambda_returns_at_first_location, n_cars_returned_at_first_location, max_n_cars_returned_at_first_location)
                    p_requests_at_second_location = poisson_probability(lambda_requests_at_second_location, n_cars_requested_at_second_location, morning_n_cars_at_second_location)
                    p_returns_at_second_location = poisson_probability(lambda_returns_at_second_location, n_cars_returned_at_second_location, max_n_cars_returned_at_second_location)
                    p = p_requests_at_first_location * p_returns_at_first_location * p_requests_at_second_location * p_returns_at_second_location
                    final_n_cars_at_first_location = morning_n_cars_at_first_location + n_cars_returned_at_first_location - n_cars_requested_at_first_location
                    final_n_cars_at_second_location = morning_n_cars_at_second_location + n_cars_returned_at_second_location - n_cars_requested_at_second_location
                    r = 10 * (n_cars_requested_at_first_location + n_cars_requested_at_second_location) - 2 * (abs(a) if a <= 0 else a - 1) - 4 * ((final_n_cars_at_first_location > 10) + (final_n_cars_at_second_location > 10))
                    new_value += p * (r + gamma * v[final_n_cars_at_first_location, final_n_cars_at_second_location])
    return new_value

def update_values(v, pi):
    for n_cars_at_first_location in range(max_n_cars_at_first_location + 1):
        for n_cars_at_second_location in range(max_n_cars_at_second_location + 1):
            a = pi[n_cars_at_first_location, n_cars_at_second_location]
            v[n_cars_at_first_location, n_cars_at_second_location] = new_state_value(n_cars_at_first_location, n_cars_at_second_location, v, a)

def update_policy(v, pi):
    for n_cars_at_first_location in range(max_n_cars_at_first_location + 1):
        for n_cars_at_second_location in range(max_n_cars_at_second_location + 1):
            possible_actions = [a for a in range(-max_n_cars_to_move, max_n_cars_to_move + 1) if a <= n_cars_at_first_location and -a <= n_cars_at_second_location]
            pi[n_cars_at_first_location, n_cars_at_second_location] = possible_actions[np.argmax([new_state_value(n_cars_at_first_location, n_cars_at_second_location, v, a) for a in possible_actions])]

# Initialization
v = np.zeros((max_n_cars_at_first_location + 1, max_n_cars_at_second_location + 1))
pi = np.zeros((max_n_cars_at_first_location + 1, max_n_cars_at_second_location + 1), dtype=int)

# Policy Evaluation
def evaluate_policy(v, pi):
    delta = float('inf')
    print('delta:', delta)
    while delta >= theta:
        old_v = np.copy(v)
        update_values(v, pi)
        delta = abs(v - old_v).max()
        print('v:\n', v[::-1])
        print('delta:', delta)

# Policy Improvement
def improve_policy(v, pi):
    policy_stable = False
    while not policy_stable:
        print('v:\n', v[::-1])
        print('pi:\n', pi[::-1])
        evaluate_policy(v, pi)
        old_pi = np.copy(pi)
        update_policy(v, pi)
        if (pi == old_pi).all():
            policy_stable = True

improve_policy(v, pi)
print('final v:\n', v[::-1])
print('final pi:\n', pi[::-1])

v:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0

v:
 [[171.76998351 182.19446905 193.00142792 203.70681686 214.0309424
  223.92446697 233.33122462 242.15121375 250.30022802 257.73699761
  264.45585001 270.47270808 275.8167221  280.52724049 284.65197384
  288.24223567 291.34285796 293.97368093 296.08512612 297.50841955
  298.11853489]
 [162.00576297 173.29884389 185.70890785 197.74041237 208.69367899
  218.75104276 228.13275472 236.86319184 244.893786   252.19270758
  258.75930028 264.61441121 269.79238103 274.33815572 278.30705478
  281.76780271 284.81769652 287.60169028 290.21327267 292.37681882
  293.53470251]
 [148.18168544 160.06068431 173.56317087 186.50982529 197.88130532
  207.99943524 217.27320238 225.81440612 233.60132659 240.6129294
  246.85840399 252.36953151 257.19198391 261.3820171  265.00745713
  268.15749045 270.97926158 273.72855216 276.62709293 279.32436269
  280.90689215]
 [136.67464201 148.52143815 161.96173784 174.83915094 186.11404136
  196.07900079 205.13244535 213.38748604 220.829404   227.44820481
  233.266803

v:
 [[267.34432036 277.59532263 288.05332184 298.40028232 308.44927283
  318.17107605 327.54774347 336.5403096  345.11662813 353.26306595
  360.97838484 368.26519545 375.12495246 381.55494812 387.54283778
  393.05285059 397.99869217 402.20794497 405.40983039 407.33373423
  408.02029059]
 [261.59021324 272.34571852 283.73349717 294.85439273 305.28731875
  315.13928064 324.55104377 333.55306454 342.13103121 350.27382484
  357.97990591 365.2514575  372.08987384 378.49303362 384.45065602
  389.93400563 394.88004513 399.17274005 402.6069532  404.86734404
  405.80241731]
 [253.2990271  264.42449019 276.49338849 288.18190033 298.90341288
  308.86440418 318.31684538 327.33756215 335.92242914 344.0604591
  351.74888111 358.98900535 365.78207054 372.12689103 378.01609797
  383.42879395 388.32343885 392.63175945 396.20360903 398.69376172
  399.80633765]
 [245.70068217 256.87438069 269.03613545 280.81169569 291.59222544
  301.59285829 311.07415421 320.11317933 328.70274198 336.82905691
  344.48750

v:
 [[327.45508746 337.60490725 347.84726428 357.93664579 367.71522696
  377.14393373 386.20972302 394.89703316 403.19780454 411.11506835
  418.65801844 425.83633312 432.65623872 439.11656832 445.20096082
  450.86059662 455.98234805 460.3495946  463.64119723 465.571448
  466.22763192]
 [323.80880731 334.27696393 345.10644204 355.68548192 365.70831474
  375.22207788 384.31440253 393.01394141 401.32419639 409.25002469
  416.80057983 423.98529657 430.81022701 437.27421363 443.36148061
  449.02623066 454.16699088 458.59675224 462.02877803 464.15146132
  464.9469937 ]
 [318.5494338  329.24833727 340.50334237 351.43862842 361.64503513
  371.23226427 380.35833418 389.08166891 397.41276766 405.35705121
  412.92319894 420.12016932 426.95371377 433.42269031 439.51200505
  445.1785946  450.3302049  454.80205106 458.33275649 460.59211734
  461.48679567]
 [313.680361   324.40956174 335.72292186 346.71354552 356.95982419
  366.5780619  375.7325298  384.48260116 392.83753656 400.80146624
  408.382101

v:
 [[366.09768746 376.18527552 386.29420732 396.22182636 405.82591051
  415.05861663 423.90683997 432.36601846 440.43929452 448.13629389
  455.46918555 462.44911498 469.08340528 475.37174441 481.29805016
  486.81284658 491.80120419 496.04407065 499.22169779 501.06279126
  501.67458171]
 [363.74500337 374.03776984 384.52520045 394.76863467 404.53037323
  413.81813676 422.68380969 431.15142024 439.23163623 446.93532622
  454.27469341 461.26077702 467.90082047 474.19451356 480.12607355
  485.64774849 490.65159672 494.93661357 498.20398625 500.16602731
  500.86486007]
 [360.35596425 370.79690307 381.55769702 392.03020033 401.91016818
  411.24552483 420.13352191 428.61745545 436.71260223 444.43028717
  451.78248947 458.78001693 465.42999109 471.73215584 477.67111654
  483.20061148 488.21793083 492.53498755 495.86759428 497.91603909
  498.67645331]
 [357.22033139 367.68067836 378.47889249 388.98698899 398.8928098
  408.24860172 417.15573055 425.65818919 433.77051317 441.50320929
  448.86761

...

v:
 [[555.03329911 560.89815197 566.05619348 570.18362477 574.65853066
  578.141593   582.12701223 584.979568   588.61612759 590.97235489
  594.344777   597.49202365 600.40510058 603.06682889 605.45060454
  607.51304791 609.18482161 610.37684801 611.03509662 611.24349322
  611.30376047]
 [552.11284701 558.1742299  563.53991287 568.03551083 572.15069258
  576.62505309 580.09771467 584.08217141 586.93102738 590.56621714
  593.9651424  597.13267885 600.06046204 602.7326152  605.12410037
  607.19314114 608.87207825 610.07345634 610.74323669 610.9617751
  611.20640358]
 [548.88723792 555.16801434 560.76384256 565.51474129 569.99308827
  574.08990773 578.56273401 582.02025185 586.00259391 589.70602405
  593.16045804 596.37031549 599.32882994 602.02298932 604.43098081
  606.51393258 608.20629613 609.42139361 610.10382706 610.64498697
  610.86117756]
 [545.42972754 551.94371483 557.78323222 562.73795515 567.47065044
  571.92837888 576.00532365 580.47571126 584.57838018 588.38514267
  591.92246

v:
 [[555.53022744 561.4010874  566.56702302 570.69025612 575.17443229
  578.653439   582.64917077 585.49796183 589.14580436 591.49894656
  594.88371879 598.04439487 600.97163139 603.64775746 606.04552491
  608.1207277  609.80296505 611.00194425 611.66297283 611.87106101
  611.9293066 ]
 [552.61698364 558.68452332 564.0583395  568.55209158 572.66654239
  577.15052224 580.62216253 584.61746124 587.46366419 591.11083526
  594.52304846 597.70494918 600.64780634 603.33522509 605.74147699
  607.82384213 609.51327909 610.72053111 611.39061217 611.60599181
  611.84590985]
 [549.3979131  555.68484651 561.28885349 566.0401433  570.52149613
  574.62276868 579.10606672 582.56686803 586.5611897  590.27809915
  593.74735255 596.97310921 599.94820314 602.65906754 605.08315052
  607.18053858 608.88405773 610.10479152 610.78616287 611.32420041
  611.53797871]
 [545.94631316 552.46629829 558.31382847 563.27010007 568.00827973
  572.47478233 576.56161744 581.04381583 585.16005094 588.98193959
  592.5357

v:
 [[556.46816768 562.3787992  567.57211556 572.23675612 576.50245163
  580.3473675  584.14778151 587.43832025 590.87377662 593.61875764
  596.76180072 599.07407377 601.96782299 604.62616227 607.02305473
  609.11154289 610.81509656 612.0349827  612.7090489  612.92080075
  612.97503527]
 [553.51683705 559.61754283 565.01478644 569.87893794 574.22436804
  578.48988303 582.32725085 586.12758182 589.41197891 592.84760526
  595.59068268 598.73379592 601.64843772 604.32328709 606.73348881
  608.83341336 610.5475615  611.77755787 612.46050172 612.6900575
  612.90309445]
 [550.26324726 556.57673833 562.20078323 567.28397091 571.86367636
  576.19849468 580.46349441 584.28948477 588.08956098 591.36493892
  594.80055653 597.99663165 600.95368119 603.66215215 606.09989507
  608.22388211 609.95993219 611.20888232 611.90537111 612.41384019
  612.63038322]
 [546.78439332 553.3246613  559.18919019 564.50226008 569.2684469
  573.83705904 578.1594462  582.42365228 586.23756785 590.03713929
  593.566349

v:
 [[556.73822973 562.65190989 567.84915379 572.51839486 576.78903381
  580.63264782 584.43787392 587.72692715 591.16663116 593.90988748
  597.05655687 599.36722069 602.26420904 604.925543   607.32529473
  609.41654911 611.12260388 612.34426461 613.0189617  613.23043765
  613.28384294]
 [553.79078333 559.89440516 565.29545916 570.16413901 574.51010912
  578.78068249 582.61932379 586.4244469  589.70937931 593.14896121
  595.89082197 599.03732965 601.95500849 604.63268979 607.04564685
  609.148261   610.86473295 612.09597044 612.77850132 613.00546247
  613.21785241]
 [550.54085194 556.85706098 562.48470683 567.57223473 572.15400829
  576.49300313 580.7633347  584.59449618 588.39933578 591.67822545
  595.11751889 598.31664044 601.27634105 603.98729908 606.42752639
  608.55400584 610.2922006  611.54206827 612.23753088 612.74539953
  612.96082911]
 [547.06544337 553.6081482  559.47596332 564.79306312 569.56178392
  574.13617683 578.46684692 582.73672725 586.55977737 590.36415766
  593.8972

v:
 [[556.95076475 562.86507027 568.06331468 572.73396542 577.0063782
  580.84725756 584.65441784 587.94109515 591.38276209 594.12410656
  597.27274064 599.58206472 602.48105001 605.14442831 607.54620118
  609.63933784 611.34694829 612.5695873  613.24453644 613.45576382
  613.50853447]
 [554.00739383 560.11163559 565.51370814 570.38384487 574.72778915
  579.00019514 582.83741311 586.64454517 589.92820492 593.36981944
  596.11013307 599.25867366 602.17846064 604.85828614 607.27335435
  609.37791012 611.09588342 612.32776739 613.00985477 613.23469263
  613.44658359]
 [550.76151173 557.07831847 562.70697009 567.79595282 572.3763109
  576.71509707 580.98742726 584.81915278 588.62617606 591.90536847
  595.34683918 598.54820655 601.51019747 604.22346068 606.66594784
  608.7944996  610.53425673 611.78464803 612.47928893 612.98574708
  613.20034308]
 [547.29011987 553.83337899 559.70215044 565.02064978 569.78818
  574.36308261 578.69570563 582.967871   586.79356591 590.60039641
  594.13602582 